In [1]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, jaccard_score

In [2]:
from nn_models.yolov5_detect import load_yolo_model, detect_objects, xywh_to_ltwh

# Тестирование моделей деткции

In [3]:
DATA = 'data/evaluation_data/train'

In [4]:
IMAGE_DIRS = [os.path.join(DATA, item, 'img1') for item in os.listdir(DATA)]

In [5]:
GT_DATA = [os.path.join(item.replace('/img1', ''), 'gt/gt.txt') for item in IMAGE_DIRS]

In [6]:
GT_DATA

['data/evaluation_data/train/MOT16-11/gt/gt.txt',
 'data/evaluation_data/train/TUD-Stadtmitte/gt/gt.txt',
 'data/evaluation_data/train/TUD-Campus/gt/gt.txt',
 'data/evaluation_data/train/MOT16-09/gt/gt.txt',
 'data/evaluation_data/train/KITTI-17/gt/gt.txt',
 'data/evaluation_data/train/PETS09-S2L1/gt/gt.txt']

In [75]:
def load_ground_truth(gt_file):
    """
    Load ground truth bounding boxes from the specified file.

    Args:
    gt_file (str): Path to the ground truth file.

    Returns:
    ground_truth (dict): Dictionary of ground truth bounding boxes per frame.
    """
    ground_truth = {}
    with open(gt_file, 'r') as f:
        for line in f:
            try:
                frame_id, _, x, y, w, h, _, _, _ = map(float, line.strip().split(','))
            except ValueError:
                frame_id, _, x, y, w, h, _, _, _, _ = map(float, line.strip().split(','))
            frame_id = int(frame_id)
            if frame_id not in ground_truth:
                ground_truth[frame_id] = []
            ground_truth[frame_id].append([x, y, w, h])
    return ground_truth

def calculate_iou(boxA, boxB):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Args:
    boxA (list): Bounding box A in format [x1, y1, w, h].
    boxB (list): Bounding box B in format [x1, y1, w, h].

    Returns:
    iou (float): Intersection over Union.
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def evaluate_detection_model(image_dir, gt_file, model_weights):
    """
    Evaluate the detection model on the dataset.

    Args:
    image_dir (str): Path to the directory with images.
    gt_file (str): Path to the ground truth file.
    model_weights (str): Path to the YOLOv5 weights file.

    Returns:
    None
    """
    model = load_yolo_model(model_weights)
    ground_truth = load_ground_truth(gt_file)

    all_gt_boxes = []
    all_pred_boxes = []
    frame_ious = []

    for frame_id, gt_boxes in tqdm(ground_truth.items()):
        image_path = os.path.join(image_dir, f'{frame_id:06d}.jpg')
        #print(image_path)
        if not os.path.exists(image_path):
            continue

        image = cv2.imread(image_path)
        results = detect_objects(model, image)

        pred_boxes = []
        for *xywh, conf, cls in results.xywh[0].cpu().numpy():
            if int(cls) == 0:  # Filter for 'person' class
                ltwh = xywh_to_ltwh(xywh)
                pred_boxes.append(ltwh)

        # Flatten lists for sklearn metrics
        for gt_box in gt_boxes:
            all_gt_boxes.append(gt_box)
        for pred_box in pred_boxes:
            all_pred_boxes.append(pred_box)

        ious = []
        for gt_box in gt_boxes:
            for pred_box in pred_boxes:
                ious.append(calculate_iou(gt_box, pred_box))

        # Calculate IoU and F1 for this frame
        if ious:
            iou = max(ious)
        else:
            iou = 0
        frame_ious.append(iou)
        #print(f'Frame {frame_id}: IoU = {iou:.4f}')
    avg_iou = np.mean(frame_ious)
    print(f'Overall IoU: {avg_iou}')
    return avg_iou

In [76]:
model_weights = 'nn_models/weights/yolo/yolov5n.pt'

In [77]:
WEIGHTS = [os.path.join('nn_models/weights/yolo', item) for item in os.listdir('nn_models/weights/yolo') if item.endswith('pt')]

In [78]:
WEIGHTS

['nn_models/weights/yolo/yolov5m.pt',
 'nn_models/weights/yolo/yolov5n.pt',
 'nn_models/weights/yolo/yolov5l.pt',
 'nn_models/weights/yolo/yolov5s.pt']

In [79]:
for model in WEIGHTS:
    print(f'evaluating model: {model}')
    model_ious = []
    for data, gt in zip(IMAGE_DIRS, GT_DATA):
        print(f'processing {data} ...')
        iou = evaluate_detection_model(data, gt, model)
        model_ious.append(iou)
    print(f'avg IoU between all videos for {model}: {np.mean(model_ious)}')

evaluating model: nn_models/weights/yolo/yolov5m.pt
processing data/evaluation_data/train/MOT16-11/img1 ...


Using cache found in /home/dtsarev/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-13 Python-3.11.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 11987MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:12<00:00, 73.21it/s]


Overall IoU: 0.943699322955675
processing data/evaluation_data/train/TUD-Stadtmitte/img1 ...


Using cache found in /home/dtsarev/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-13 Python-3.11.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 11987MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 179/179 [00:01<00:00, 155.82it/s]


Overall IoU: 0.8849566087252204
processing data/evaluation_data/train/TUD-Campus/img1 ...


...

Overall IoU: 0.9146625331328243
processing data/evaluation_data/train/KITTI-17/img1 ...


Using cache found in /home/dtsarev/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-13 Python-3.11.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 11987MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 145/145 [00:00<00:00, 182.95it/s]


Overall IoU: 0.8518640503069613
processing data/evaluation_data/train/PETS09-S2L1/img1 ...


Using cache found in /home/dtsarev/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-13 Python-3.11.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 11987MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 795/795 [00:04<00:00, 166.31it/s]

Overall IoU: 0.8635482452739085
avg IoU between all videos for nn_models/weights/yolo/yolov5s.pt: 0.8815053122428219


# Тестирование REID модели

In [7]:
import os
import cv2
import torch
import numpy as np
import random
from PIL import Image
from tqdm import tqdm
from scipy.spatial.distance import cosine
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nn_models.reid_model import load_reid_model

2024-07-14 18:29:02.158514: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-14 18:29:02.185032: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-14 18:29:02.732674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
from nn_models.reid_model import ReIDModel, load_reid_model

In [29]:
def load_ground_truth(gt_file):
    """
    Load ground truth bounding boxes from the specified file.

    Args:
    gt_file (str): Path to the ground truth file.

    Returns:
    ground_truth (dict): Dictionary of ground truth bounding boxes per frame.
    """
    ground_truth = {}
    with open(gt_file, 'r') as f:
        for line in f:
            try:
                frame_id, track_id, x, y, w, h, _, _, _, _= map(float, line.strip().split(','))
            except ValueError:
                frame_id, track_id, x, y, w, h, _, _, _= map(float, line.strip().split(','))
            frame_id = int(frame_id)
            track_id = int(track_id)
            if frame_id not in ground_truth:
                ground_truth[frame_id] = []
            ground_truth[frame_id].append([track_id, x, y, w, h])
    return ground_truth

def extract_bounding_boxes(image_dir, ground_truth, num_boxes=100):
    """
    Extract bounding boxes from images based on ground truth data.

    Args:
    image_dir (str): Path to the directory with images.
    ground_truth (dict): Dictionary of ground truth bounding boxes per frame.
    num_boxes (int): Number of bounding boxes to extract.

    Returns:
    extracted_images (dict): Dictionary of extracted images per track ID.
    """
    extracted_images = {}
    for frame_id, gt_boxes in ground_truth.items():
        image_path = os.path.join(image_dir, f'{frame_id:06d}.jpg')
        if not os.path.exists(image_path):
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: Image at path {image_path} could not be loaded.")
            continue

        for gt_box in gt_boxes:
            track_id, x, y, w, h = gt_box
            x, y, w, h = int(x), int(y), int(w), int(h)

            # Validate bounding box coordinates
            if x < 0 or y < 0 or x + w > image.shape[1] or y + h > image.shape[0]:
                #print(f"Warning: Bounding box {x, y, w, h} is out of image bounds {image.shape}. Skipping this bounding box.")
                continue

            bbox_img = image[y:y + h, x:x + w]
            #cv2.imshow('test', bbox_img)
            #cv2.waitKey(0)
            if bbox_img.size == 0:
                #print(f"Warning: Bounding box {x, y, w, h} resulted in an empty image. Skipping this bounding box.")
                continue

            bbox_img = cv2.cvtColor(bbox_img, cv2.COLOR_BGR2RGB)
            bbox_img_pil = Image.fromarray(bbox_img)

            if track_id not in extracted_images:
                extracted_images[track_id] = []
            extracted_images[track_id].append(bbox_img_pil)

    # Randomly select num_boxes // 2 bounding boxes with the same ID and num_boxes // 2 with different IDs
    random.seed(42)
    selected_boxes = []
    track_ids = list(extracted_images.keys())

    for _ in range(num_boxes // 2):
        track_id = random.choice(track_ids)
        if len(extracted_images[track_id]) > 1:
            selected_boxes.append((random.choice(extracted_images[track_id]), track_id))
    
    for _ in range(num_boxes // 2):
        track_id_1, track_id_2 = random.sample(track_ids, 2)
        selected_boxes.append((random.choice(extracted_images[track_id_1]), track_id_2))

    return selected_boxes

def evaluate_reid_model(image_dir, gt_file, model_type, num_boxes=1000, threshold=0.18):
    """
    Evaluate the Re-ID model on the dataset.

    Args:
    image_dir (str): Path to the directory with images.
    gt_file (str): Path to the ground truth file.
    model_type (str): Type of the Re-ID model.
    num_boxes (int): Number of bounding boxes to evaluate.
    threshold (float): Cosine distance threshold.

    Returns:
    None
    """
    model = load_reid_model(model_type)
    ground_truth = load_ground_truth(gt_file)
    selected_boxes = extract_bounding_boxes(image_dir, ground_truth, num_boxes)

    features = []
    labels = []

    for img, track_id in tqdm(selected_boxes):
        feature = model.extract_features(img)
        features.append(feature)
        labels.append(track_id)

    y_true = []
    y_pred = []

    for i in range(0, len(features), 2):
        feat1, feat2 = features[i], features[i+1]
        label1, label2 = labels[i], labels[i+1]

        dist = cosine(feat1.flatten(), feat2.flatten())
        is_same = label1 == label2
        #print(is_same, dist)
        y_true.append(1 if is_same else 0)
        y_pred.append(1 if dist < threshold else 0)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    
    return accuracy, precision, recall, f1

In [30]:
MODELS = ['osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25']

In [32]:
for model in MODELS:
    print(f'evaluating model: {model}')
    model_metrics = []
    for data, gt in zip(IMAGE_DIRS, GT_DATA):
        print(f'processing {data} ...')
        metrics = evaluate_reid_model(data, gt, model)
        model_metrics.append(metrics)
    print(f'avg metrics between all videos for {model}: {np.mean(model_metrics, axis=0)}')

evaluating model: osnet_x1_0
processing data/evaluation_data/train/MOT16-11/img1 ...
***  osnet_x1_0
Successfully loaded imagenet pretrained weights from "/home/dtsarev/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 198.76it/s]


Accuracy: 0.9820
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
processing data/evaluation_data/train/TUD-Stadtmitte/img1 ...
***  osnet_x1_0
Successfully loaded imagenet pretrained weights from "/home/dtsarev/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 204.28it/s]


Accuracy: 0.8820
Precision: 0.4054
Recall: 0.2885
F1 Score: 0.3371
processing data/evaluation_data/train/TUD-Campus/img1 ...
***  osnet_x1_0
Successfully loaded imagenet pretrained weights from "/home/dtsarev/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"


...

Accuracy: 0.9660
Precision: 0.1429
Recall: 0.0833
F1 Score: 0.1053
processing data/evaluation_data/train/KITTI-17/img1 ...
***  osnet_x0_25
Successfully loaded imagenet pretrained weights from "/home/dtsarev/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 210.59it/s]


Accuracy: 0.8540
Precision: 0.2222
Recall: 0.1034
F1 Score: 0.1412
processing data/evaluation_data/train/PETS09-S2L1/img1 ...
***  osnet_x0_25
Successfully loaded imagenet pretrained weights from "/home/dtsarev/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 210.96it/s]


Accuracy: 0.9300
Precision: 0.1429
Recall: 0.0800
F1 Score: 0.1026
avg metrics between all videos for osnet_x0_25: [0.89366667 0.26375865 0.18963874 0.21233629]


Как можно видеть, на основании F1 score, osnet_x0_25 подходит лучше всего. 